# Check processed corpus

## Check for duplicates

In [9]:
# Load white supremacist corpus
import pandas as pd

path = '../tmp/white_supremacist_train_corpus.pkl'
ws_data = pd.read_pickle(path)
ws_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 229305 entries, calderon2021_0 to stormfront_99972
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   text        229305 non-null  object             
 1   word_count  229305 non-null  int64              
 2   dataset     229305 non-null  object             
 3   source      229305 non-null  object             
 4   domain      229305 non-null  object             
 5   timestamp   229051 non-null  datetime64[ns, UTC]
 6   num_words   229305 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(4)
memory usage: 14.0+ MB


In [10]:
ws_data.text.duplicated(keep='first').sum()

4480

In [14]:
ws_data[ws_data.duplicated(['text'], keep=False)].sort_values('text')

,text,word_count,dataset,source,domain,timestamp,num_words
id,,,,,,,
qian2018_9389,""" #jihadiaap "" is now trending at rank 4 in india",13,qian2018,twitter,tweet,2015-02-06 18:53:38+00:00,11
elsherief2021_41,""" #jihadiaap "" is now trending at rank 4 in india",13,elsherief2021,twitter,tweet,2015-02-06 18:53:38+00:00,11
qian2018_36177,""" : 51 % of immigrant households on welfare while only 30 % ' native americans ' are",18,qian2018,twitter,tweet,2015-09-04 06:37:58+00:00,18
qian2018_40213,""" : 51 % of immigrant households on welfare while only 30 % ' native americans ' are",20,qian2018,twitter,tweet,2015-09-04 06:48:12+00:00,18
qian2018_20628,""" : can't wait to be in germany next year """,11,qian2018,twitter,tweet,2015-02-03 19:22:20+00:00,11
...,...,...,...,...,...,...,...
jokubausaite2020_448872,卐 british natsoc ϟϟ -- -- -- jonathan bowden : sir oswald mosley :,14,jokubausaite2020,4chan,forum,2017-02-10 18:09:31+00:00,14
jokubausaite2020_470233,卐 british natsoc ϟϟ -- -- -- jonathan bowden : sir oswald mosley :,14,jokubausaite2020,4chan,forum,2017-04-18 02:11:46+00:00,14
jokubausaite2020_437080,"卐 sieg heil ! 卐 thread for discussion of national socialism , fascism , traditionalism , and european identity movements . share links , pdfs , reading , videos , and propaganda . 卐 - report in for your country ! 卐 - take back the board ! 卐 - drive out degeneracy ! 卐 if you do not see a general in the catalog please copy from the pastebin and create one ! - help spread the love ! 卐 卐 pastebin - 卐 卐 take the surveys ! show them we belong ! show your strength ! 卐 - news , radio , commentary & forums 卐 - viral media 卐 - physical media",116,jokubausaite2020,4chan,forum,2016-11-28 18:25:43+00:00,116


## Check short posts in white supremacist corpus to see if express the ideology

In [ ]:
# Load white supremacist corpus
import pandas as pd

path = '../tmp/white_supremacist_train_corpus.pkl'
ws_data = pd.read_pickle(path)
ws_data.info()

In [ ]:
# View short posts across domains
from IPython.display import display

pd.set_option('display.max_colwidth', None)

for domain in data.domain.unique():
    print(domain)
    domain_data = data.query('domain==@domain')
    short = domain_data[domain_data.text.str.split().str.len() <= 10]
    display(short.sample(10))

In [ ]:
# View distribution of post length overall
import plotly.express as px
pd.options.plotting.backend = 'plotly'

lengths = data.text.str.split().str.len()

In [ ]:
data['length'] = lengths

In [ ]:
# remove outliers
sampled = lengths.sample(int(1e6))
sampled[sampled < 200].hist()

In [ ]:
s.domain.value_counts()

In [ ]:
# Plot lengths across domains
s = data[data.length < 100].sample(int(1e6))
# s.hist(column='length')
px.histogram(s, x='length', color='domain', barmode='overlay')

In [ ]:
# Plot lengths across non-forum domains
s = data.query('length < 100 and domain != "forum"')
# s.hist(column='length')
px.histogram(s, x='length', color='domain', barmode='overlay')

In [ ]:
cdf = lengths.value_counts().sort_index().cumsum()/len(lengths)
cdf

In [ ]:
px.line(cdf[:100])

In [ ]:
# cdfs = {domain: data[data.domain==domain]['length'].value_counts().sort_index().cumsum()/len(data) for domain in data.domain.unique()}
# cdfs = pd.concat([pd.DataFrame(
#     {'cumsum': data[data.domain==domain]['length'].value_counts().sort_index().cumsum()/len(data), 'domain': domain}) for domain in data.domain.unique()
#                  ])
cdfs = pd.concat([pd.DataFrame(
    data[data.domain==domain]['length'].value_counts().sort_index().cumsum()/len(data[data.domain==domain])).assign(domain=domain).rename_axis('value').reset_index() for domain in data.domain.unique()
                 ])
cdfs.info()
cdfs[cdfs.value <= 100]

In [ ]:
px.line(cdfs[cdfs.value <= 100], x='value', y='length', color='domain')

## Check Patriot Front chat posts for white supremacist ideology

In [ ]:
'172809042299715584'

In [6]:
# Load white supremacist corpus
import pandas as pd

path = '../tmp/white_supremacist_train_corpus.pkl'
data = pd.read_pickle(path)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 229305 entries, calderon2021_0 to stormfront_99972
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   text        229305 non-null  object             
 1   word_count  229305 non-null  int64              
 2   dataset     229305 non-null  object             
 3   source      229305 non-null  object             
 4   domain      229305 non-null  object             
 5   timestamp   229051 non-null  datetime64[ns, UTC]
 6   num_words   229305 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(4)
memory usage: 14.0+ MB


In [8]:
pd.set_option('display.max_colwidth', None)
data.query('domain=="chat"').sample(30)[['text', 'dataset']]

,text,source
id,,
patriotfront_18048,`` [ redacted ] said his friends told him the people in the car the shots were fired from shouted “ sieg heil ” after asking them if they had seen someone named “ kyle. ” ``,discord
patriotfront_12392,nigger being interviewed who was inside speech `` i did n't even know it was going on . professor shoved me in line to get a ticked and i yelled blm '',discord
patriotfront_14115,it 'll help if you have the doctor write something explaining the situation effects of the drug,discord
patriotfront_16345,when we win i vote the this clown not be ovened fucker is too hilarious to not have walking around providing free entertainment .,discord
patriotfront_18029,"well , the website is a bit more bare bones and additions/edits have to be done directly through html . so ... maybe not for the time being .",discord
patriotfront_2068,make sure your daughters dont share the same fate as those in sa .,discord
patriotfront_11402,"so , i think it will take a while for his emotional response to die down .",discord
patriotfront_12870,"i have n't spoken to kevin yet , it just slipped my mind , but i did talk to < @ ! 154153868593201153 > about it when we were flyering . i 'm going to buy the canvas either tomorrow or the next day after work . i 'll either buy that or make them out of cardboard . those seem more likely < @ 363070228864696330 >",discord
patriotfront_16111,i do n't even understand what that guy is trying to say,discord


# Corpus stats

In [10]:
# Load data
import pandas as pd

path = '../tmp/white_supremacist_train_corpus.pkl'
# path = '../tmp/antiracist_train_corpus.pkl'
# path = '../tmp/domain_test_corpus.pkl'
ws_data = pd.read_pickle(path)
# path = '../data/corpora/white_supremacist_corpus.json'
# data = pd.read_json(path, orient='table')
# print(len(data))
# data.columns
ws_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 229305 entries, calderon2021_0 to stormfront_99972
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   text        229305 non-null  object             
 1   word_count  229305 non-null  int64              
 2   dataset     229305 non-null  object             
 3   source      229305 non-null  object             
 4   domain      229305 non-null  object             
 5   timestamp   229051 non-null  datetime64[ns, UTC]
 6   num_words   229305 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(4)
memory usage: 14.0+ MB


In [4]:
# Post and word counts total and per domain
# data['num_words'] = data.text.str.split().str.len()
stats = data.groupby('domain').agg({'num_words': ['count', 'sum', 'mean']})
stats.columns = ['post_count', 'word_count', 'avg_post_length']
stats

total = pd.DataFrame({'post_count': len(data), 'word_count': data.num_words.sum(), 'avg_post_length': data.num_words.mean()}, index=['total'])
total

pd.concat([stats,total])

,post_count,word_count,avg_post_length
chat,18739,425654,22.714873
forum,100000,6511295,65.112950
long-form,38036,11420987,300.267825
tweet,72530,1268597,17.490652
total,229305,19626533,85.591387


# Compare corpora

In [1]:
# Load white supremacist corpus
import pandas as pd

path = '../tmp/white_supremacist_train_corpus.pkl'
ws_data = pd.read_pickle(path)
ws_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 229305 entries, calderon2021_0 to stormfront_99972
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   text        229305 non-null  object             
 1   word_count  229305 non-null  int64              
 2   dataset     229305 non-null  object             
 3   source      229305 non-null  object             
 4   domain      229305 non-null  object             
 5   timestamp   229051 non-null  datetime64[ns, UTC]
 6   num_words   229305 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(4)
memory usage: 14.0+ MB


In [2]:
# Load comparison corpora
path = '../tmp/neutral_train_corpus.pkl'
neutral_data = pd.read_pickle(path)
neutral_data.info()

# Post and word counts total and per domain
# data['num_words'] = data.text.str.split().str.len()
stats = neutral_data.groupby('domain').agg({'num_words': ['count', 'sum', 'mean']})
stats.columns = ['post_count', 'word_count', 'avg_post_length']
stats

total = pd.DataFrame({'post_count': len(neutral_data), 'word_count': neutral_data.num_words.sum(), 'avg_post_length': neutral_data.num_words.mean()}, index=['total'])
total

pd.concat([stats,total])

<class 'pandas.core.frame.DataFrame'>
Index: 298412 entries, discord_match_99825 to twitter_match_89106
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   text        298412 non-null  object             
 1   word_count  298412 non-null  int64              
 2   dataset     298412 non-null  object             
 3   source      298412 non-null  object             
 4   domain      298412 non-null  object             
 5   timestamp   210015 non-null  datetime64[ns, UTC]
 6   num_words   298412 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(4)
memory usage: 18.2+ MB


,post_count,word_count,avg_post_length
chat,88397,466080,5.272577
forum,84271,3708288,44.004319
long-form,36639,32119081,876.636398
tweet,89105,1667438,18.713181
total,298412,37960887,127.209653


In [3]:
# Load comparison corpora
path = '../tmp/antiracist_train_corpus.pkl'
antiracist_data = pd.read_pickle(path)
antiracist_data.info()

# Post and word counts total and per domain
# data['num_words'] = data.text.str.split().str.len()
stats = antiracist_data.groupby('domain').agg({'num_words': ['count', 'sum', 'mean']})
stats.columns = ['post_count', 'word_count', 'avg_post_length']
stats

total = pd.DataFrame({'post_count': len(antiracist_data), 'word_count': antiracist_data.num_words.sum(), 'avg_post_length': antiracist_data.num_words.mean()}, index=['total'])
total

pd.concat([stats,total])

<class 'pandas.core.frame.DataFrame'>
Index: 200681 entries, reddit_antiracist_0 to medium_antiracist_929
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   text        200681 non-null  object             
 1   word_count  200681 non-null  int64              
 2   dataset     200681 non-null  object             
 3   source      200681 non-null  object             
 4   domain      200681 non-null  object             
 5   timestamp   200680 non-null  datetime64[ns, UTC]
 6   num_words   200681 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(4)
memory usage: 12.2+ MB


,post_count,word_count,avg_post_length
forum,90731,4002747,44.116642
long-form,38036,31965244,840.394468
tweet,71914,1224819,17.031718
total,200681,37192810,185.332991


In [4]:
data = pd.concat([ws_data, neutral_data, antiracist_data], keys=['white_supremacist', 'neutral', 'antiracist'],
                names=['corpus', 'id']).reset_index(level='corpus')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 728398 entries, calderon2021_0 to medium_antiracist_929
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   corpus      728398 non-null  object             
 1   text        728398 non-null  object             
 2   word_count  728398 non-null  int64              
 3   dataset     728398 non-null  object             
 4   source      728398 non-null  object             
 5   domain      728398 non-null  object             
 6   timestamp   639746 non-null  datetime64[ns, UTC]
 7   num_words   728398 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(5)
memory usage: 50.0+ MB


In [ ]:
import plotly.express as px

fig = px.box(data[data.word_count<100], x='corpus', y='word_count')
fig.update_traces(boxpoints=False)

In [ ]:
pd.options.plotting.backend = 'plotly'
selected = neutral_data.query('domain=="forum"')
selected[selected.word_count<200].plot.hist(x='word_count')

In [11]:
data.dataset.value_counts()

papasavva2020       1901408
stormfront           706232
jokubausaite2020     415921
ironmarch            148003
qian2018              66294
calderon2021          25685
patriotfront          18739
pruden2022            12351
elsherief2021          4489
Name: dataset, dtype: int64

In [ ]:
data.groupby(['domain', 'dataset'])['text'].count()

In [ ]:
# counts = data.groupby(['domain', 'dataset'])['text'].count().reset_index()
counts = data.groupby(['domain', 'dataset'])['text'].count()
counts

In [ ]:
data.groupby(['domain', 'dataset', 'source'])['text'].count()

In [ ]:
# Can't figure out how to sort wihin domains
# counts.sort_values(['domain', 'dataset'], ascending=False).groupby('domain').groups()
# counts.sort_values(['domain', 'dataset'], ascending=False).groupby('domain')['text'].sum()

In [ ]:
# Num of posts from each dataset (maybe should do log scale)
pd.options.plotting.backend = 'plotly'
vc = data.dataset.value_counts()
print(vc)
vc.plot.bar(text=vc.values)

In [ ]:
# Num of posts from each dataset (maybe should do log scale)
pd.options.plotting.backend = 'plotly'
vc = data.dataset.value_counts()
print(vc)
vc.plot.bar(text=vc.values)

In [ ]:
# Number of words
data['num_words'] = data.text.str.split().str.len()
# data.head()

word_count =  data.groupby('dataset')['num_words'].sum()
display(word_count)
word_count.sort_values(ascending=False).plot.bar().show()

# Total posts and words
print(f'{len(data)} posts')
print(f'{word_count.sum()} words')

In [ ]:
import plotly.express as px

# By domain
dataset_info = pd.DataFrame([
    {'dataset': 'qian2018', 'domain': 'tweet/short propaganda'},
    {'dataset': 'elsherief2021', 'domain': 'tweet/short propaganda'},
    {'dataset': 'degibert2019', 'domain': 'forum'},
    {'dataset': 'patriotfront', 'domain': 'chat'},
    {'dataset': 'alatawi2021', 'domain': 'tweet/short propaganda'},
    {'dataset': 'adl_heatmap', 'domain': 'tweet/short propaganda'},
    {'dataset': 'ironmarch', 'domain': 'forum'},
    {'dataset': '4chan', 'domain': 'forum'},
    {'dataset': 'stormfront', 'domain': 'forum'},
    {'dataset': 'calderon2021', 'domain': 'long-form'},
    {'dataset': 'pruden2022', 'domain': 'long-form'},
]).set_index('dataset')
dataset_info

# data.join(dataset_info, on='dataset').domain.value_counts().plot.bar(
#     title="Number of posts by domain", 
#     labels=dict(index='domain', value='number of posts'),
# )

vc = data.join(dataset_info, on='dataset').domain.value_counts()
fig = px.bar(vc, title="Number of posts by domain", 
       labels=dict(index='domain', value='number of posts'),
        text = [f'{val}<br>{val/vc.sum(): .1%}' for val in vc.values],
    )
fig.update_layout(showlegend=False)

In [ ]:
num_words = data.join(dataset_info, on='dataset').groupby('domain')['num_words'].sum().sort_values(ascending=False)
fig = px.bar(num_words,
    title="Number of words by domain", 
    labels=dict(index='domain', value='number of words'),
    text = [f'{val}<br>{val/num_words.sum(): .1%}' for val in num_words.values],
       # log_y=True
    )
fig.update_layout(showlegend=False)

In [ ]:
# Investigate date ranges
print(len(data))
print(data.timestamp.count())

In [ ]:
pd.options.plotting.backend = 'plotly'
data.timestamp.sample(int(1e6)).hist()
# data.timestamp.plot.hist()

In [ ]:
start_date = pd.Timestamp(year=2000, month=1, day=1, tz='utc')
data.query('timestamp > @start_date').timestamp.sample(int(1e5)).hist()

In [ ]:
# Bin or group by month
import plotly.express as px
# pd.options.plotting.backend = 'plotly'

start_date = pd.Timestamp(year=2000, month=1, day=1, tz='utc')
merged =  data.join(dataset_info, on='dataset').query('timestamp > @start_date').sample(int(1e6))
# merged.hist(column='timestamp', by='domain')
# merged.hist(column=['timestamp'])
px.histogram(merged, x='timestamp', color='domain', barmode=')

In [ ]:
# Bin or group by month
px.histogram(merged, x='timestamp', color='domain', barmode='overlay')

In [ ]:
import plotly.express as px

px.histogram(data.query('domain=="forum"').sample(int(1e5)), x='timestamp', color='dataset', barmode='overlay')